# Prompt length and complexity management with language models

In this notebook, we explore how to manage prompt length and complexity when working with LLMs. Two core challenges often arise in AI interactions: ensuring prompts are clear and concise, and handling long or complex text inputs that might exceed the model's token limit. By managing these factors efficiently, we can enhance the quality and relevance of model responses, and ensure that even lengthy or multifaceted queries can be processed effectively.


In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

## Balancing detail and conciseness in prompts
It is often a challenge to decide how much detail to include in a prompt. More detail can improve precision, but too much can overwhelm or limit the model. It is important to find the right balance between providing enough context to guide the model while avoiding excessive verbosity that could confuse or overburden the model.

Let’s compare two approaches: one detailed, one concise—for the same topic.

In [3]:
# A detailed prompt template
detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition,
    historical context, key components, practical applications, and any relevant examples.
    Also, discuss any controversies or debates surrounding the topic, and mention potential
    future developments or trends."""
)

# A concise prompt template
concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance."
)

# Evaluate on the same topic
topic = "artificial intelligence"

# Generate responses from both prompts
print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)

Detailed response:
### Definition of Artificial Intelligence

Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types:

1. **Narrow AI**: This is AI that is designed and trained for a specific task, such as voice recognition, image processing, or playing chess.
2. **General AI**: This refers to a type of AI that possesses the ability to understand, learn, and apply intelligence across a wide range of tasks, similar to human cognitive abilities. General AI remains largely theoretical and has not yet been achieved.

### Historical Context

The concept of AI dates back to ancient history, but it formally began in the mid-20th century. Key milestones include:

- **1950s**: Alan Turing proposed the 

Here we construct two `PromptTemplate` objects with different levels of verbosity and query the same topic. The output will help us evaluate trade-offs in coverage and clarity.


## Analysis of prompt balance
Now that we have generated responses from the detailed and concise prompts, we can analyze the differences in the generated answers. We will compare them based on their information coverage, clarity, and potential use cases, and suggest strategies for finding the right balance between detail and conciseness in future prompts.

In [4]:
# Analyze the differences between detailed and concise responses
analysis_prompt = PromptTemplate(
    input_variables=["detailed_response", "concise_response"],
    template="""Compare the following two responses on artificial intelligence:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

# Generate both versions again
detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

# Run the comparative analysis
analysis = llm.invoke(analysis_prompt.format(
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

### Analysis of Differences

#### 1. Information Coverage
- **Detailed Response**: This response provides a comprehensive overview of AI, covering its definition, historical context, key components, practical applications, ethical controversies, future developments, and trends. It delves into the evolution of AI, detailing milestones and significant breakthroughs, making it an extensive resource for those seeking in-depth knowledge. The response also mentions various domains where AI is applied, ethical issues, and future considerations, providing a well-rounded perspective.
  
- **Concise Response**: The concise response offers a high-level summary of AI, defining it and mentioning its processes, categories (narrow and general AI), and overarching importance. It briefly highlights potential applications across various sectors but lacks the depth and detailed historical context found in the detailed response. While it touches on the transformative potential of AI, it does not address e

The model acts as a meta-evaluator, helping us reflect on when and why we might prefer detailed or brief prompts, and how to strike the right balance in our application. We define an analysis prompt that takes both the detailed and concise responses as inputs and the model suggests strategies for balancing the two types of prompts.


##Strategies for handling long contexts
Handling long contexts, such as long documents or multiple-step tasks, can exceed the token limits of language models. To address this, we explore three strategies: chunking, summarization, and iterative processing. These techniques can help process large text inputs efficiently while retaining important details.

### 1. Chunking
When dealing with long documents, one effective approach is to break them down into smaller, manageable chunks. This ensures that each chunk is within the token limit and can be processed independently.

In [5]:
# A long passage about artificial intelligence, its history, applications, and future prospects...
long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration.
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter with chunk size and overlap configuration
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Maximum chunk size in characters
    chunk_overlap=200,  # Overlap between chunks to maintain context
    length_function=len  # Function to calculate the length of text
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


We use the `RecursiveCharacterTextSplitter` to divide a long text into chunks. This utility ensures that the chunks don’t exceed model input limits while maintaining coherence by using overlapping content.

### 2. Summarization
Summarization can be used to condense long texts while retaining key information. Let's use LangChain's summarization chain to demonstrate this.

In [6]:
# Convert text chunks to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result['output_text'])

Summary:
Artificial intelligence (AI) is a field of computer science aimed at creating machines that replicate human cognition. Since its inception in the 1950s, AI has progressed through key developments like neural networks and machine learning, enabling applications in areas such as speech recognition and autonomous vehicles. While AI holds promise for transforming industries, it raises concerns about job displacement, algorithmic bias, and ethics. Future advancements are expected to emphasize explainability, ethics, and human-AI collaboration, necessitating careful management of its societal impacts to ensure responsible innovation.


We apply a map-reduce summarization approach that processes chunks in parallel, then combines the outputs into a coherent summary. This is ideal for working with longer texts efficiently.

### 3. Iterative processing
For tasks that require multiple steps of analysis or processing, iterative processing allows us to handle each step sequentially. This method ensures that complex tasks are broken down into smaller, manageable parts. Let's demonstrate this with a multi-step analysis task.

In [7]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.

    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.

    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

# Define the analysis steps
analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

# Perform the iterative analysis
final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
The text offers a comprehensive overview of artificial intelligence (AI), covering its definition, historical development, and various subfields. It highlights practical applications across multiple industries, discusses ongoing debates surrounding ethical considerations and societal impacts, and addresses the challenges faced by researchers and developers. Importantly, the text emphasizes the need for responsible innovation as AI continues to evolve and integrate with other technologies.

**Conclusion**: Overall, the text underscores the transformative potential of AI while also acknowledging the complexities and ethical dilemmas associated with its advancement. It calls for a balanced approach that prioritizes societal benefits and ethical responsibility, reflecting a nuanced understanding of AI's future implications.


We define a function iterative_analysis that processes the text through multiple steps, such as identifying topics, summarizing key points, and providing a conclusion. The function applies each step sequentially, updating the text after each analysis.

Here we simulate a pipeline that mimics how a human might analyze complex content step by step. This technique is useful for document review, report writing, and case analysis.

## Practical tips for nanaging prompt length and complexity
Let's conclude with some practical tips for managing prompt length and complexity in real-world applications.

In [8]:
# Generate practical tips for handling prompt length and complexity
tips_prompt = """
Based on the examples and strategies we've explored for managing prompt length and complexity,
provide a list of 5 practical tips for developers working with large language models.
Each tip should be concise and actionable.
"""

# Generate the tips
tips = llm.invoke(tips_prompt).content
print(tips)

Here are five practical tips for developers working with large language models:

1. **Chunk Input Data**: Break down large inputs into smaller, manageable chunks. This helps the model process information more efficiently and reduces the risk of exceeding token limits.

2. **Use Clear and Specific Prompts**: Formulate prompts that are direct and unambiguous. Clearly define the task or question to minimize confusion and improve the relevance of the model's responses.

3. **Iterative Refinement**: Start with a broad prompt and iteratively refine your queries based on the model's outputs. This approach helps hone in on the desired responses without overwhelming the model.

4. **Leverage Contextual Cues**: Provide context where necessary by including relevant information or examples in the prompt. This can improve the model's understanding and output quality.

5. **Monitor Token Usage**: Keep track of token counts in both inputs and outputs to avoid hitting limits. Use tools or libraries th

Here, we use a prompt to ask the model for five practical tips on managing prompt length and complexity. This gives us a distilled list of best practices that we can apply in production scenarios or prompt design workflows.